In [4]:
!wget http://cvdl.cupoy.com/file-download/part01/box.png
!wget http://cvdl.cupoy.com/file-download/part01/box_in_scene.png
!wget http://cvdl.cupoy.com/file-download/part01/lena.png

--2019-12-10 09:26:57--  http://cvdl.cupoy.com/file-download/part01/box.png
Resolving cvdl.cupoy.com (cvdl.cupoy.com)... 13.35.5.45, 13.35.5.49, 13.35.5.97, ...
Connecting to cvdl.cupoy.com (cvdl.cupoy.com)|13.35.5.45|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50728 (50K) [image/png]
Saving to: ‘box.png’

box.png             100%[===================>]  49.54K  --.-KB/s    in 0.003s  

2019-12-10 09:26:57 (14.8 MB/s) - ‘box.png’ saved [50728/50728]

--2019-12-10 09:27:00--  http://cvdl.cupoy.com/file-download/part01/box_in_scene.png
Resolving cvdl.cupoy.com (cvdl.cupoy.com)... 13.35.5.45, 13.35.5.49, 13.35.5.97, ...
Connecting to cvdl.cupoy.com (cvdl.cupoy.com)|13.35.5.45|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 122490 (120K) [image/png]
Saving to: ‘box_in_scene.png’

box_in_scene.png    100%[===================>] 119.62K   320KB/s    in 0.4s    

2019-12-10 09:27:00 (320 KB/s) - ‘box_in_scene.png’ saved [122490/122490]

--20

# 作業

實作本篇提到的三大概念

- 翻轉：實作上下左右的翻轉
- 縮放：比較鄰近差值與雙立方插值 (或雙線性插值) 的圖片品質
- 平移：建立 Translation Transformation Matrix 來做平移

In [0]:
import cv2
import time
import numpy as np

img_path = 'lena.png'
img = cv2.imread(img_path)

## 上下左右翻轉圖片

In [0]:
# 水平翻轉 (horizontal)
img_hflip = img[:, ::-1, :]

# 垂直翻轉 (vertical)
img_vflip = img[::-1, :, :]

# 水平 + 垂直翻轉
img_hvflip = img[::-1,::-1, :]

# 組合 + 顯示圖片
hflip = np.hstack((img, img_hflip))
vflip = np.hstack((img_vflip, img_hvflip))
img_flip = np.vstack((hflip, vflip))
while True:
    cv2.imwrite('hflip_image.png', hflip)
    cv2.imwrite('vflip_image.png', vflip)
    cv2.imwrite('img_flip_image.png', img_flip)
    break

## 縮放圖片

### 放大

我們先透過縮小圖片去壓縮原有圖片保有的資訊，再放大比較不同方法之間的速度與圖片品質

In [15]:
# 將圖片縮小成原本的 20%
img_test = cv2.resize(img, None, fx=0.2, fy=0.2)

# 將圖片放大為"小圖片"的 8 倍大 = 原圖的 1.6 倍大
fx, fy = 8, 8

# 鄰近差值 scale + 計算花費時間
start_time = time.time()
img_area_scale = cv2.resize(img_test, None, fx=fx, fy=fy, interpolation=cv2.INTER_NEAREST)
print('INTER_NEAREST zoom cost {}'.format(time.time() - start_time))

# 雙立方差補 scale + 計算花費時間
start_time = time.time()
img_cubic_scale = cv2.resize(img_test, None, fx=fx, fy=fy, interpolation=cv2.INTER_CUBIC)
print('INTER_CUBIC zoom cost {}'.format(time.time() - start_time))

# 組合 + 顯示圖片
img_zoom = np.hstack((img_area_scale, img_cubic_scale))
while True:
    cv2.imwrite('zoom_image.png', img_zoom)
    break

INTER_NEAREST zoom cost 0.0050144195556640625
INTER_CUBIC zoom cost 0.004426002502441406


## 平移幾何轉換

In [0]:
# 設定 translation transformation matrix
# x 平移 50 pixel; y 平移 100 pixel
M = np.array([[1, 0, 50],
              [0, 1, 100]], dtype=np.float32)
shift_img = cv2.warpAffine(img, M, (img.shape[1], img.shape[0]))

# 組合 + 顯示圖片
img_shift = np.hstack((img, shift_img))
while True:
    cv2.imwrite('shift_image.png', img_shift)
    break